<a href="https://colab.research.google.com/github/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/VGG19_Diagnosis_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multimodal Template

In [ ]:
# import sys
# IN_COLAB = 'google.colab' in sys.modules

# if IN_COLAB:
#     !pip install deriva
#     !pip install bdbag
#     !pip install --upgrade --force pydantic
#     !pip install git+https://github.com/informatics-isi-edu/deriva-ml git+https://github.com/informatics-isi-edu/eye-ai-ml

In [1]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))

In [2]:
# Prerequisites

import json
import os
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging
# import torch

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [3]:

from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "eye-ai" #@param
host = 'www.eye-ai.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

2024-05-29 15:55:25,340 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-05-29 15:55:25,341 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2024-05-29 15:55:25,343 - INFO - Setting up RefreshTokenAuthorizer with auth_client=[instance:140092464299600]
2024-05-29 15:55:25,344 - INFO - Setting up a RenewingAuthorizer. It will use an auth type of Bearer and can handle 401s.
2024-05-29 15:55:25,345 - INFO - RenewingAuthorizer will start by using access_token with hash "8d1f9520c9018abaab6ca103dff23b1ef1ec0b1bfc2fe67a9248ef62314db3cf"
2024-05-29 15:55:25,345 - INFO - Executing token refresh without client credentials
2024-05-29 15:55:25,346 - INFO - Fetching new token from Globus Auth
2024-05-29 15:55:25,734 - INFO - request done (success)
2024-05-29 15:55:25,734 - INFO - RenewingAuthorizer.

You are already logged in.


Connect to Eye-AI catalog.  Configure to store data local cache and working directories.  Initialize Eye-AI for pending execution based on the provided configuration file.

In [5]:
# Variables to configure the rest of the notebook.

cache_dir = '/data'        # Directory in which to cache materialized BDBags for datasets
working_dir = '/data'    # Directory in which to place output files for later upload.

configuration_rid="2-BDA8" # rid



In [6]:
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2024-05-29 15:55:48,362 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-05-29 15:55:48,363 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


In [7]:
# @title Initiate an Execution
configuration_records = EA.execution_init(configuration_rid=configuration_rid)
configuration_records.model_dump()

2024-05-29 15:55:52,764 - INFO - File [/data/lizhiwei/EyeAI_working/Execution_Metadata/Execution_Config-multimodal_template.json] transfer successful. 0.68 KB transferred. Elapsed time: 0:00:00.000056.
2024-05-29 15:55:52,765 - INFO - Verifying MD5 checksum for downloaded file [/data/lizhiwei/EyeAI_working/Execution_Metadata/Execution_Config-multimodal_template.json]
2024-05-29 15:55:52,784 - INFO - Configuration validation successful!
2024-05-29 15:55:54,153 - INFO - Attempting to resolve http://identifiers.org/minid:10aYGORSTY9ah into a valid set of URLs.
2024-05-29 15:55:54,945 - INFO - The identifier minid:10aYGORSTY9ah resolved into the following locations: [https://eye-ai-shared.s3.amazonaws.com//6656e08709b2f7c0bbed8284fdfb550d/2024-05-24_15.56.36/Dataset_2-7P5P.zip]
2024-05-29 15:55:54,949 - INFO - Attempting GET from URL: https://eye-ai-shared.s3.amazonaws.com//6656e08709b2f7c0bbed8284fdfb550d/2024-05-24_15.56.36/Dataset_2-7P5P.zip
2024-05-29 15:55:55,124 - INFO - File [/data/

{'caching_dir': PosixPath('/data'),
 'working_dir': PosixPath('/data/lizhiwei/EyeAI_working'),
 'vocabs': {'Workflow_Type': [{'name': 'multimodal_data_analysis',
    'rid': '2-BDAC'}],
  'Execution_Asset_Type': [{'name': 'Multimodal_Analysis', 'rid': '2-BDAE'}]},
 'execution_rid': '2-BDAA',
 'workflow_rid': '2-BDAG',
 'bag_paths': [PosixPath('/data/2-7P5P_a14456c09cb78569ae70c30ee54767a9f0b3a4aa0ae9f80871dd9f8ff3f6f3dc/Dataset_2-7P5P')],
 'assets_paths': [],
 'configuration_path': PosixPath('/data/lizhiwei/EyeAI_working/Execution_Metadata/Execution_Config-multimodal_template.json')}

In [20]:
# View data

# subject = pd.read_csv(configuration_records.bag_paths[0]/'data/Subject.csv')
# subject

# observation = pd.read_csv(configuration_records.bag_paths[0]/'data/Observation.csv')
# observation

# clinic = pd.read_csv(configuration_records.bag_paths[0]/'data/Clinical_Records.csv')
# clinic

# observation_clinic_asso = pd.read_csv(configuration_records.bag_paths[0]/'data/Observation_Clinic_Asso.csv')
# observation_clinic_asso # association table between observation table and clinic record table

# icd10 = pd.read_csv(configuration_records.bag_paths[0]/'data/Clinic_ICD10.csv')
# icd10

# icd10_asso = pd.read_csv(configuration_records.bag_paths[0]/'data/Clinic_ICD_Asso.csv')
# icd10_asso # association table between clinic record table and ICD10 code

# report = pd.read_csv(configuration_records.bag_paths[0]/'data/Report.csv')
# report

# RNFL_OCR = pd.read_csv(configuration_records.bag_paths[0]/'data/RNFL_OCR.csv')
# RNFL_OCR

HVF_OCR = pd.read_csv(configuration_records.bag_paths[0]/'data/HVF_OCR.csv')
HVF_OCR


,RID,RCT,RMT,RCB,RMB,Field_Size,Strategy,False_Neg,False_Pos,Fixation_Loss,...,Visual_Acuity,GHT,PSD.1,Abs_Perc,Abs_Val,Pat_Perc,Pat_Val,Raw_Val,Report,Image_Side_Vocab
0,2-B93Y,2024-05-20 23:15:48.182905+00,2024-05-20 23:15:48.182905+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,24-2,SITA Standard,5%,7%,5/15,...,20/30,Outside Normal Limits,5.19,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...,2-ATX0,2SK2
1,2-B940,2024-05-20 23:15:48.182905+00,2024-05-20 23:15:48.182905+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,24-2,SITA Fast,Off,0%,NaN,...,NaN,Outside Normal Limits,7.85,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...,2-ATX8,2SK2
2,2-B942,2024-05-20 23:15:48.182905+00,2024-05-20 23:15:48.182905+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,24-2,SITA Standard,0%,0%,0/15,...,NaN,Outside Normal Limits,5.12,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...,2-ATXG,2SK2
3,2-B944,2024-05-20 23:15:48.182905+00,2024-05-20 23:15:48.182905+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,24-2,SITA Standard,0%,1%,0/15,...,20/30,Within Normal Limits,NaN,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...,2-ATXM,2SK2
4,2-B946,2024-05-20 23:15:48.182905+00,2024-05-20 23:15:48.182905+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,24-2,SITA Fast,0%,0%,0/11,...,20/20,Within Normal Limits,NaN,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...,2-ATXR,2SK2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2144,2-BD9Y,2024-05-20 23:59:49.438927+00,2024-05-20 23:59:49.438927+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,24-2,SITA Fast,Off,3%,NaN,...,20/30,Outside Normal Limits,865.00,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...,2-B2E6,2SK0
2145,2-BDA0,2024-05-20 23:59:49.438927+00,2024-05-20 23:59:49.438927+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,24-2,SITA Fast,0%,17%XX,1/10,...,20/25,Within Normal Limits,NaN,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...,2-B2EA,2SK0
2146,2-BDA2,2024-05-20 23:59:49.438927+00,2024-05-20 23:59:49.438927+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,24-2,SITA Standard,0%,0%,NaN,...,NaN,Within Normal Limits,NaN,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...,2-B2EM,2SK0
2147,2-BDA4,2024-05-20 23:59:49.438927+00,2024-05-20 23:59:49.438927+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,24-2,SITA Fast,Off,0%,NaN,...,20/30,Outside Normal Limits,2.19,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...,2-B2EW,2SK0


In [ ]:
# @title Execute Training algorithm
from eye_ai.models.vgg19_hyper_parameter_tuning import main #import the new logistic module.
with EA.execution(execution_rid=configuration_records.execution_rid) as exec:
  main()


In [ ]:
# @title Save Execution Assets (model) and Metadata
uploaded_assets = EA.execution_upload(configuration_records.execution_rid, False)

# 